In [24]:
_ARREST_KEY = "ARREST_KEY"

_ARREST_BORO = "ARREST_BORO"
_JURISDICTION_CODE = "JURISDICTION_CODE"
_OFFENSE_LEVEL = "LAW_CAT_CD"

_ARREST_DATE = "ARREST_DATE"
_ARREST_MONTH = "ARREST_MONTH" 

_ARREST_PRECINCT = "ARREST_PRECINCT"

_PERPETRATOR_RACE = "PERP_RACE"
_PERPETRATOR_SEX = "PERP_SEX"
_PERPETRATOR_AGE_GROUP="AGE_GROUP"

_INTERNAL_CLASSIFICATION="PD_CD"
_INTERNAL_CLASSIFICATION_DESCRIPTION="PD_DESC"
_THREE_DIGIT_INTERNAL_CLASSIFICATION="KY_CD"

_LAW_CODE="LAW_CODE"

_GEOGRAPHICAL_POSITION = "New Georeferenced Column"

_NYS_X_COORD = "X_COORD_CD"
_NYS_Y_COORD = "Y_COORD_CD"

_LATITUDE='Latitude'
_LONGITUDE='Longitude'


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [26]:
df = pd.read_csv('./cleaned_data.csv')
df.head()


,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,Latitude,Longitude,ARREST_MONTH
0,FELONY ASSAULT,PL-1211200,F,Q,105,0,40.737043,-73.735514,January
1,FELONY ASSAULT,PL-1200502,F,B,48,0,40.855109,-73.892818,March
2,FELONY ASSAULT,PL-1200512,F,S,121,0,40.628967,-74.163275,May
3,FELONY ASSAULT,PL-1211200,F,Q,100,0,40.591980,-73.800066,June
4,RAPE,PL-1302503,F,M,14,0,40.753533,-73.994537,January


In [27]:
df[_ARREST_PRECINCT] = df[_ARREST_PRECINCT].astype('object')
df[_JURISDICTION_CODE] = df[_JURISDICTION_CODE].astype('object')

In [28]:
df_num= df.select_dtypes(exclude='object')
df_obj= df.select_dtypes(include='object')

df_num.info()
df_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188682 entries, 0 to 188681
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Latitude   188682 non-null  float64
 1   Longitude  188682 non-null  float64
dtypes: float64(2)
memory usage: 2.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188682 entries, 0 to 188681
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   OFNS_DESC          188682 non-null  object
 1   LAW_CODE           188682 non-null  object
 2   LAW_CAT_CD         188682 non-null  object
 3   ARREST_BORO        188682 non-null  object
 4   ARREST_PRECINCT    188682 non-null  object
 5   JURISDICTION_CODE  188682 non-null  object
 6   ARREST_MONTH       188682 non-null  object
dtypes: object(7)
memory usage: 10.1+ MB


In [29]:
df_obj[_OFFENSE_LEVEL].unique()
#df_obj=df_obj.drop(columns=["OFNS_DESC","LAW_CODE"])# drop some columns to deal with out of mem error
#above features have major impact on accuracy.

array(['F', 'M', 'V'], dtype=object)

In [30]:
df_obj.head()

,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,ARREST_MONTH
0,FELONY ASSAULT,PL-1211200,F,Q,105,0,January
1,FELONY ASSAULT,PL-1200502,F,B,48,0,March
2,FELONY ASSAULT,PL-1200512,F,S,121,0,May
3,FELONY ASSAULT,PL-1211200,F,Q,100,0,June
4,RAPE,PL-1302503,F,M,14,0,January


In [31]:
df_obj= pd.get_dummies(df_obj, drop_first=True)

target = (df_obj['LAW_CAT_CD_M'])
df_obj = df_obj.drop(columns=['LAW_CAT_CD_M'],axis=1)
df_obj

,OFNS_DESC_ADMINISTRATIVE CODES,OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW,OFNS_DESC_ANTICIPATORY OFFENSES,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLAR'S TOOLS,OFNS_DESC_BURGLARY,OFNS_DESC_CANNABIS RELATED OFFENSES,OFNS_DESC_CHILD ABANDONMENT/NON SUPPORT 1,OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF,...,JURISDICTION_CODE_1,JURISDICTION_CODE_2,ARREST_MONTH_August,ARREST_MONTH_February,ARREST_MONTH_January,ARREST_MONTH_July,ARREST_MONTH_June,ARREST_MONTH_March,ARREST_MONTH_May,ARREST_MONTH_September
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188677,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
188678,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
188679,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
188680,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True


In [32]:
attr = pd.concat([df_obj,df_num],axis=1)


print(attr.shape[1])
attr.head()

1104


,OFNS_DESC_ADMINISTRATIVE CODES,OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW,OFNS_DESC_ANTICIPATORY OFFENSES,OFNS_DESC_ARSON,OFNS_DESC_ASSAULT 3 & RELATED OFFENSES,OFNS_DESC_BURGLAR'S TOOLS,OFNS_DESC_BURGLARY,OFNS_DESC_CANNABIS RELATED OFFENSES,OFNS_DESC_CHILD ABANDONMENT/NON SUPPORT 1,OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF,...,ARREST_MONTH_August,ARREST_MONTH_February,ARREST_MONTH_January,ARREST_MONTH_July,ARREST_MONTH_June,ARREST_MONTH_March,ARREST_MONTH_May,ARREST_MONTH_September,Latitude,Longitude
0,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,40.737043,-73.735514
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,40.855109,-73.892818
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,40.628967,-74.163275
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,40.591980,-73.800066
4,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,40.753533,-73.994537


In [33]:
target

0         False
1         False
2         False
3         False
4         False
          ...  
188677     True
188678     True
188679    False
188680     True
188681    False
Name: LAW_CAT_CD_M, Length: 188682, dtype: bool

In [34]:
from sklearn.model_selection import train_test_split

# Split the data into training and test
attr_train, attr_test, target_train, target_test = train_test_split(attr, target, test_size=0.40, random_state=5)

In [35]:
from sklearn.compose import make_column_selector

cat_selector = make_column_selector(dtype_include=bool)
num_selector = make_column_selector(dtype_include=np.number)
num_selector(attr_train)

['Latitude', 'Longitude']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(
            handle_unknown="infrequent_if_exist"
            ), df_obj.columns.tolist()),
        ('num', StandardScaler(), df_num.columns.tolist()),
    ]
    #,     remainder='passthrough'
)

In [37]:
#CART
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline


model = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',DecisionTreeClassifier())
])



In [38]:

model.fit(attr_train,target_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat', OneHotEncoder(),
                                                  ['OFNS_DESC_ADMINISTRATIVE '
                                                   'CODES',
                                                   'OFNS_DESC_ALCOHOLIC '
                                                   'BEVERAGE CONTROL LAW',
                                                   'OFNS_DESC_ANTICIPATORY '
                                                   'OFFENSES',
                                                   'OFNS_DESC_ARSON',
                                                   'OFNS_DESC_ASSAULT 3 & '
                                                   'RELATED OFFENSES',
                                                   "OFNS_DESC_BURGLAR'S TOOLS",
                                                   'OFNS_DESC_BURGLARY',
                                                   'OFNS_DESC_CANNABIS RELATED '
                                                   'OFFENSES',
                                                   'OFNS_DE...
                                                   'OFNS_DESC_GRAND LARCENY OF '
                                                   'MOTOR VEHICLE',
                                                   'OFNS_DESC_HARRASSMENT 2',
                                                   'OFNS_DESC_HOMICIDE-NEGLIGENT,UNCLASSIFIE',
                                                   'OFNS_DESC_HOMICIDE-NEGLIGENT-VEHICLE',
                                                   'OFNS_DESC_INTOXICATED & '
                                                   'IMPAIRED DRIVING',
                                                   'OFNS_DESC_INTOXICATED/IMPAIRED '
                                                   'DRIVING',
                                                   'OFNS_DESC_JOSTLING', ...]),
                                                 ('num', StandardScaler(),
                                                  ['Latitude', 'Longitude'])])),
                ('classifier', DecisionTreeClassifier())])

In [39]:
target_pred = model.predict(attr_test)

ValueError: Found unknown categories [True] in column 18 during transform

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
accuracy = accuracy_score(target_test, target_pred)
print(f'Accuracy: {accuracy}')
print()

print(confusion_matrix(target_test,target_pred))
print(classification_report(target_test,target_pred))

Accuracy: 0.9997615041140541

[[32914     3]
 [   15 42541]]
              precision    recall  f1-score   support

       False       1.00      1.00      1.00     32917
        True       1.00      1.00      1.00     42556

    accuracy                           1.00     75473
   macro avg       1.00      1.00      1.00     75473
weighted avg       1.00      1.00      1.00     75473



In [ ]:
attr.columns

Index(['OFNS_DESC_ADMINISTRATIVE CODES',
       'OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW',
       'OFNS_DESC_ANTICIPATORY OFFENSES', 'OFNS_DESC_ARSON',
       'OFNS_DESC_ASSAULT 3 & RELATED OFFENSES', 'OFNS_DESC_BURGLAR'S TOOLS',
       'OFNS_DESC_BURGLARY', 'OFNS_DESC_CANNABIS RELATED OFFENSES',
       'OFNS_DESC_CHILD ABANDONMENT/NON SUPPORT 1',
       'OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF',
       ...
       'ARREST_MONTH_August', 'ARREST_MONTH_February', 'ARREST_MONTH_January',
       'ARREST_MONTH_July', 'ARREST_MONTH_June', 'ARREST_MONTH_March',
       'ARREST_MONTH_May', 'ARREST_MONTH_September', 'Latitude', 'Longitude'],
      dtype='object', length=1104)

In [ ]:
df_num.columns

Index(['Latitude', 'Longitude'], dtype='object')

In [ ]:
df_obj.columns

Index(['OFNS_DESC_ADMINISTRATIVE CODES',
       'OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW',
       'OFNS_DESC_ANTICIPATORY OFFENSES', 'OFNS_DESC_ARSON',
       'OFNS_DESC_ASSAULT 3 & RELATED OFFENSES', 'OFNS_DESC_BURGLAR'S TOOLS',
       'OFNS_DESC_BURGLARY', 'OFNS_DESC_CANNABIS RELATED OFFENSES',
       'OFNS_DESC_CHILD ABANDONMENT/NON SUPPORT 1',
       'OFNS_DESC_CRIMINAL MISCHIEF & RELATED OF',
       ...
       'JURISDICTION_CODE_1', 'JURISDICTION_CODE_2', 'ARREST_MONTH_August',
       'ARREST_MONTH_February', 'ARREST_MONTH_January', 'ARREST_MONTH_July',
       'ARREST_MONTH_June', 'ARREST_MONTH_March', 'ARREST_MONTH_May',
       'ARREST_MONTH_September'],
      dtype='object', length=1102)

In [ ]:
model.named_steps

{'preprocessor': ColumnTransformer(transformers=[('cat',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                  ['OFNS_DESC_ADMINISTRATIVE CODES',
                                   'OFNS_DESC_ALCOHOLIC BEVERAGE CONTROL LAW',
                                   'OFNS_DESC_ANTICIPATORY OFFENSES',
                                   'OFNS_DESC_ARSON',
                                   'OFNS_DESC_ASSAULT 3 & RELATED OFFENSES',
                                   "OFNS_DESC_BURGLAR'S TOOLS",
                                   'OFNS_DESC_BURGLARY',
                                   'OFNS_DESC_CANNABIS RELATED OFFENSES',
                                   'OFNS_DESC_CHI...
                                   'OFNS_DESC_GAMBLING',
                                   'OFNS_DESC_GRAND LARCENY',
                                   'OFNS_DESC_GRAND LARCENY OF MOTOR VEHICLE',
                                   'OFNS_DESC_HARRASSMENT 2',

In [ ]:
# I don't think there's a need to plot this. There are too many features.
classifier = model.named_steps['classifier']
print(classifier)

DecisionTreeClassifier()
